In [1]:
include("../src/Utero.jl")
using .Utero

In [2]:
x = Utero.Tracker(7.0)
Utero.ForwardBackward(x -> x*x + x, x)

⬅ctx.Gradients = Dict{Any, Any}(Main.Utero.var"#1#2"() => 1.0)
⬅ctx.Gradients = Dict{Any, Any}(Main.Utero.var"#1#2"() => 2.0)
⬅ctx.Gradients = Dict{Any, Any}(Main.Utero.var"#1#2"() => 4.0)
⬅ctx.Gradients = Dict{Any, Any}(Main.Utero.var"#1#2"() => 56.0)


Dict{Any, Any} with 1 entry:
  #1 => 784.0

In [9]:
# should be 42
Utero.ForwardBackward(x -> x^2.0 + x^2.0 + x*x, Utero.Tracker(7.0)) 

MethodError: MethodError: no method matching ^(::Main.Utero.Tracker{Float64}, ::Int64)

Closest candidates are:
  ^(!Matched::Union{AbstractChar, AbstractString}, ::Integer)
   @ Base strings/basic.jl:733
  ^(!Matched::Complex{<:AbstractFloat}, ::Integer)
   @ Base complex.jl:869
  ^(!Matched::Complex{<:Integer}, ::Integer)
   @ Base complex.jl:871
  ...


In [4]:
# Gradient should be 56.21
Utero.ForwardBackward(w -> 2.0 * (((2.0*w + 12.0) - w) * w) + sin(w^2.0), Utero.Param(7.0, 33))

UndefVarError: UndefVarError: `Param` not defined

# Differentiation for Scalar Parameters

In [5]:
x = 12 # treated as constant 

Params = IdDict()
w = Utero.Tracked(7.0, Params)
grads = Utero.GetGradient(_ -> 2 * (((2*w + x) - w) * w) + sin(w^2), Params) 
# should be approx. 56.21
println(grads)

Params = IdDict()
w = Utero.Tracked(25.0, Params)
grads = Utero.GetGradient(_ -> sin(w)^2, Params)
# should be approx. -0.26
println(grads)

Params = IdDict()
w1 = Utero.Tracked(13.2, Params); w2 = Utero.Tracked(89.0, Params)
grads = Utero.GetGradient(_ -> (cos(w2)^3 * 4sin(w1))^2 + cos(w1*w2)^2, Params) 
# should be approx. (27.53, 3.0428)
println(grads)


UndefVarError: UndefVarError: `Tracked` not defined

# Differentiation for Tensor Parameters

In [6]:

Parameters = IdDict()
w1 = Utero.Tracked(rand(2, 2), Parameters)
w2 = Utero.Tracked(rand(2, 2), Parameters)
w3 = Utero.Tracked(rand(2, 2), Parameters)

grads = Utero.GetGradient(x -> prod(*(w3, *(w2, *(w1, x)))), rand(2,1), Parameters)

UndefVarError: UndefVarError: `Tracked` not defined

The product of the parameter matrices should match A.

In [7]:
A = rand(2,2)
X = rand(100, 2)
Y = mapslices(v -> (A * v), X, dims=(2))


println(A)
println((w1 * w2 * w3).val)

for (x, y) in zip(eachrow(X), eachrow(Y))
    x = vec(x); y = vec(y)
    grads = Utero.GetGradient(x -> Utero.mse(*(w3, *(w2, *(w1, x))), y), x, Parameters)
    Utero.GradientStep(0.01, Parameters, grads)
end

println("Now, they should match")
println(A)
println((w1 * w2 * w3).val)

[0.5084750306343131 0.3176377626012572; 0.4016571699839002 0.3903676152040998]


UndefVarError: UndefVarError: `w1` not defined